In [4]:
import re
from lightrag.core import Component, Generator
from lightrag.components.model_client import OpenAIClient
from lightrag.components.model_client import GroqAPIClient
from lightrag.utils import setup_env # make sure you have a .env file with OPENAI_API_KEY and GROQ_API_KEY

In [5]:
template_doc = r"""<SYS> You are a doctor </SYS> User: {{input_str}}"""

Let's turn on the library log to help with debugging.

In [6]:
from lightrag.utils import enable_library_logging
enable_library_logging()

In [7]:
#Toy example

class DocQA(Component):
    def __init__(self):
        super(DocQA, self).__init__()
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )

    def call(self, query: str) -> str:
        return self.doc(prompt_kwargs={"input_str": query}).data
    

In [8]:
doc = DocQA()
doc

2024-06-10 10:05:19 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
)

In [9]:
print(doc("What is the best treatment for headache?"))

2024-06-10 10:05:19 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?'}
2024-06-10 10:05:19 - INFO - [generator.py:195:call] - model_kwargs: {}
2024-06-10 10:05:19 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?'}]}
2024-06-10 10:05:22 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-10 10:05:22 - INFO - [generator.py:203:call] - output: GeneratorOutput(data='As a doctor, the best treatment for a headache depends on the underlying cause of the headache. In general, some common treatments for headaches include:\n\n1. Over-the-counter pain relievers such as acetaminophen, ibuprofen, or aspirin.\n2. Getting enough rest and relaxation.\n3. Applying a cold or warm compress to the head or neck.\n4. Hy

In [10]:
# list other subcomponents

for subcomponent in doc.named_components():
    print(subcomponent)

('', DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
))
('doc', Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
  (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
  (model_client): OpenAIClient()
))
('doc.prompt', Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str']))
('doc.model_client', OpenAIClient())


Let's add a parameter

In [11]:
from lightrag.core.parameter import Parameter

doc.register_parameter("demo", param=Parameter(data="demo"))

In [12]:
# list all parameters
for param in doc.named_parameters():
    print(param)

('demo', Parameter: demo)


In [13]:
doc.to_dict()

{'type': 'DocQA',
 'data': {'_components': {'doc': {'type': 'Generator',
    'data': {'_components': {'model_client': {'type': 'OpenAIClient',
       'data': {'_components': {},
        '_parameters': {},
        'training': False,
        'sync_client': <openai.OpenAI at 0x12766e490>,
        'async_client': None,
        '_api_key': None}},
      'prompt': {'type': 'Prompt',
       'data': {'_components': {},
        '_parameters': {},
        'training': False,
        '_template_string': '<SYS> You are a doctor </SYS> User: {{input_str}}',
        'prompt_variables': ['input_str'],
        'preset_prompt_kwargs': None}}},
     '_parameters': {},
     'training': False,
     'template': '<SYS> You are a doctor </SYS> User: {{input_str}}',
     'preset_prompt_kwargs': None,
     'model_kwargs': {'model': 'gpt-3.5-turbo'},
     'output_processors': None,
     '_trainable_params': []}}},
  '_parameters': {'demo': {'data': 'demo', 'requires_opt': True}},
  'training': False}}

In [14]:
from lightrag.utils.serialization import save_json

save_json(doc.to_dict(), "doc.json")

In [15]:
doc.state_dict()

OrderedDict([('demo', Parameter: demo)])

In [16]:
doc.call("What is the best treatment for a cold?")

2024-06-10 10:05:22 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'What is the best treatment for a cold?'}
2024-06-10 10:05:22 - INFO - [generator.py:195:call] - model_kwargs: {}
2024-06-10 10:05:22 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for a cold?'}]}
2024-06-10 10:05:25 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-10 10:05:25 - INFO - [generator.py:203:call] - output: GeneratorOutput(data='The best treatment for a cold typically includes rest, staying hydrated, and using over-the-counter medications to help relieve symptoms such as fever, congestion, and sore throat. It is also important to practice good hygiene by washing your hands frequently and covering your nose and mouth when you cough or sneeze to prevent the spread of the

'The best treatment for a cold typically includes rest, staying hydrated, and using over-the-counter medications to help relieve symptoms such as fever, congestion, and sore throat. It is also important to practice good hygiene by washing your hands frequently and covering your nose and mouth when you cough or sneeze to prevent the spread of the virus. If your symptoms persist or worsen, it is recommended to consult with a healthcare provider.'

In [17]:
from lightrag.core.component import FunComponent

def add_one(x):
    return x + 1

fun_component = FunComponent(add_one)
print(fun_component(1))  
print(type(fun_component))  

# output:
# 2
# <class 'core.component.FunComponent'>

2
<class 'lightrag.core.component.FunComponent'>


In [18]:
from lightrag.core.component import fun_to_component 

fun_component = fun_to_component(add_one)
print(fun_component(1))
print(type(fun_component))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [19]:
# use it as a decorator
@fun_to_component
def add_one(x):
    return x + 1

print(add_one(1))
print(type(add_one))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [20]:
from lightrag.core.component import Sequential

@fun_to_component
def enhance_query(query:str) -> str:
    return query + "Please be concise and only list the top treatments."

seq = Sequential(enhance_query, doc)

query = "What is the best treatment for headache?"
print(seq(query))

2024-06-10 10:05:25 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?Please be concise and only list the top treatments.'}
2024-06-10 10:05:25 - INFO - [generator.py:195:call] - model_kwargs: {}
2024-06-10 10:05:25 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?Please be concise and only list the top treatments.'}]}
2024-06-10 10:05:26 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-10 10:05:26 - INFO - [generator.py:203:call] - output: GeneratorOutput(data='1. Over-the-counter pain relievers such as ibuprofen or acetaminophen\n2. Rest and hydration\n3. Relaxation techniques such as deep breathing or meditation', error=None, raw_response='1. Over-the-counter pain relievers such as ibuprofen or 

In [23]:
# sequential with just a function, will raise error
# def enhance_query(query:str) -> str:
#     return query + "Please be concise and only list the top treatments."
# seq2 = Sequential(enhance_query, doc)
# print(seq2(query))
# print(seq2)

In [21]:
seq

Sequential(
  (0): EnhanceQueryComponent()
  (1): DocQA(
    (doc): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
      (model_client): OpenAIClient()
    )
  )
)

# TODO: LLM for single choices